# SAKI Homework 4 SS21

## Implemention of a reinforcement loearning based algorith for a warehouse organizing robot

## Package import

In [1]:
import numpy as np
import pandas as pd
import csv
import itertools
import mdptoolbox
import os
import copy

## Data loading

In [2]:
current_working_directory = os.getcwd()
path_to_warehouse_training_file = os.path.join(current_working_directory, "warehousetraining.txt")
path_to_warehouse_order_file = os.path.join(current_working_directory, 'warehouseorder.txt')
warehouse_training = pd.read_csv(path_to_warehouse_training_file, names=["action", "color"], delimiter="\t")
warehouse_order = pd.read_csv(path_to_warehouse_order_file, names=["action", "color"], delimiter="\t")

## Exploration of the data set to examine the probabilities of the occurances of colors and actions

In [3]:
num_training_samples = len(warehouse_training)
probability_evaluation = warehouse_training.groupby(['action','color']).size().reset_index(name='num')
probability_evaluation['probability'] = probability_evaluation['num']/num_training_samples
probability_evaluation

,action,color,num,probability
0,restore,blue,995,0.121683
1,restore,red,2064,0.252415
2,restore,white,1029,0.125841
3,store,blue,995,0.121683
4,store,red,2064,0.252415
5,store,white,1030,0.125963


## Definition of the 6 actions in our warehouse

### Definition of actions
We have three colors in our warehouse and two actions (store and restore).

In [4]:
list_of_actions = ['store_red', 'store_blue', 'store_white', 'restore_red','restore_blue', 'restore_white']

### Definition of States

In [5]:
items = ['red', 'blue','white', 'empty']
state_matrix = list(itertools.product(items,items, items, items, list_of_actions))

In [6]:
state_matrix

[('red', 'red', 'red', 'red', 'store_red'),
 ('red', 'red', 'red', 'red', 'store_blue'),
 ('red', 'red', 'red', 'red', 'store_white'),
 ('red', 'red', 'red', 'red', 'restore_red'),
 ('red', 'red', 'red', 'red', 'restore_blue'),
 ('red', 'red', 'red', 'red', 'restore_white'),
 ('red', 'red', 'red', 'blue', 'store_red'),
 ('red', 'red', 'red', 'blue', 'store_blue'),
 ('red', 'red', 'red', 'blue', 'store_white'),
 ('red', 'red', 'red', 'blue', 'restore_red'),
 ('red', 'red', 'red', 'blue', 'restore_blue'),
 ('red', 'red', 'red', 'blue', 'restore_white'),
 ('red', 'red', 'red', 'white', 'store_red'),
 ('red', 'red', 'red', 'white', 'store_blue'),
 ('red', 'red', 'red', 'white', 'store_white'),
 ('red', 'red', 'red', 'white', 'restore_red'),
 ('red', 'red', 'red', 'white', 'restore_blue'),
 ('red', 'red', 'red', 'white', 'restore_white'),
 ('red', 'red', 'red', 'empty', 'store_red'),
 ('red', 'red', 'red', 'empty', 'store_blue'),
 ('red', 'red', 'red', 'empty', 'store_white'),
 ('red', 'red

### Definition of th model
Our model consists of four cells, as we have a 2x2 warehouse matrix. Every cell is indexed by the numbers from 0 to 3.

In [7]:
cell_index = np.arange(0,4)
cell_index

array([0, 1, 2, 3])

The costs are defined based on the distance the roboter has to walk to the cell. We have a 2x2 field and the robot cannot walk diagonally. Thus, the minimum cost is 1 and the maximum one is 3.

In [8]:
distance_costs = [1,2,2,3]

### Definition of the tranbsition probability matrix
Based on the probabilities of the occurances of the actions in the dataset, we create a transition probability matrix. It indicates the probability to change from one state to another based on the distribution of the training data set. 

In [9]:
probability_store_red = probability_evaluation.loc[(probability_evaluation['color'] == 'red') & (probability_evaluation['action'] == 'store'), 'probability'].item()
probability_store_blue = probability_evaluation.loc[(probability_evaluation['color'] == 'blue') & (probability_evaluation['action'] == 'store'), 'probability'].item()
probability_store_white = probability_evaluation.loc[(probability_evaluation['color'] == 'white') & (probability_evaluation['action'] == 'store'), 'probability'].item()
probability_restore_red = probability_evaluation.loc[(probability_evaluation['color'] == 'red') & (probability_evaluation['action'] == 'restore'), 'probability'].item()
probability_restore_blue = probability_evaluation.loc[(probability_evaluation['color'] == 'blue') & (probability_evaluation['action'] == 'restore'), 'probability'].item()
probability_restore_white = probability_evaluation.loc[(probability_evaluation['color'] == 'white') & (probability_evaluation['action'] == 'restore'), 'probability'].item()

In [10]:
def is_valid_transation(cell_id, current_state, next_state):
    if current_state[-1] == 'restore_red' or current_state[-1] == 'restore_blue' or current_state[-1] == 'restore_white':
        if next_state[-1][cell_id] == 'empty' and current_state[-1][cell_id] == current_state[-1].split('_')[1]:
            return True
            #if np.count_nonzero(current_state[-1] == next_state)==3:
            #    return True
    elif current_state[-1] == 'store_red' or current_state[-1] == 'store_blue' or current_state[-1] == 'store_white':
        if next_state[-1][cell_id] == current_state[-1].split('_')[1] and current_state[-1][cell_id] == 'empty': 
            #if np.count_nonzero(current_state[-1] == next_state)==3:
            return True
    return False

If there is a row consisting only of zeros, we will set a 100% = 1 transition probability to stay in the current state. Moreover, we normalize the probabilities of entering in another state, so that these sum up to 1.

In [11]:
def normalize(matrix):
    for index, row in enumerate(matrix):
        if np.sum(row) == 0:
            matrix[index, index] = 1
            continue

    corrected_matrix =matrix / matrix.sum(axis=1)[:, np.newaxis]
    return corrected_matrix

In [12]:
tpm_matrix = []

for cell_id in cell_index:
    tpm_per_cell = np.zeros((len(state_matrix), len(state_matrix)), dtype=np.float16)
    for i, current_state in enumerate(state_matrix):
        for j, next_state in enumerate(state_matrix):
            if is_valid_transation(cell_id, current_state, next_state):
                next_action = next_state[4]
                if next_action == 'store_red':
                    tpm_per_cell[i, j] = probability_store_red
                elif next_action == 'store_blue':
                    tpm_per_cell[i, j] = probability_store_blue
                elif next_action == 'store_white':
                    tpm_per_cell[i, j] = probability_store_white
                elif next_action == 'restore_red':
                    tpm_per_cell[i, j] = probability_restore_red
                elif next_action == 'restore_white':
                    tpm_per_cell[i, j] = probability_restore_white
                else:
                    tpm_per_cell[i, j] = probability_restore_blue        
    tpm_matrix.append(normalize(tpm_per_cell))
    

### Definition of the reward matrix

In [13]:
rewards = [3,2,2,1]
def create_reward_matrix():

    reward_matrix = np.zeros((len(state_matrix), len(cell_index)))
    for cell_id in cell_index:
        for i, state in enumerate(state_matrix):
            if state[-1].split('_')[0] == 'restore':
                #restoring if color is suiting
                if state[cell_id] == state[-1].split('_')[1]:
                    reward_matrix[i, cell_id] = rewards[cell_id] 
                else:
                    reward_matrix[i, cell_id] = -10
            elif state[-1].split('_')[0] == 'store':
                # Storing possible if empty cell there
                if state[cell_id] =='empty':
                    reward_matrix[i, cell_id] = rewards[cell_id]
                else:
                    reward_matrix[i, cell_id] = -10
    return reward_matrix

In [14]:
reward_matrix = create_reward_matrix()

### Greedy approach
If neither storing nor restoring is possible, costs of distance_costs[0] = 1 are added to the total costs.

In [15]:
def greedy_helper(current_warehouse):
    current_warehouse = copy.deepcopy(current_warehouse)
    result = current_warehouse[:4]
    task = current_warehouse[4]
    color = current_warehouse[-1]
    if task =='store':
        for i, field in enumerate(result):
            if field=='empty':
                current_warehouse[i]=color
                return current_warehouse, i

    elif task == 'restore':
        for i, field in enumerate(result):
            if field == color:
                current_warehouse[i]='empty'
                return current_warehouse, i
    return current_warehouse, 0

In [16]:
def greedy_distance(data_set):
    greedy_warehouse = ['empty', 'empty', 'empty','empty','-','-']
    total_reward = 0
    greedy_warehouse_states=list()
    
    for i, data_row in data_set.iterrows():
        greedy_warehouse[4] = data_row.action
        greedy_warehouse[5] = data_row.color
        
        greedy_warehouse, action = greedy_helper(greedy_warehouse)
        greedy_warehouse_states.append(tuple(greedy_warehouse))
        total_reward = total_reward + (distance_costs[action]*2)
        
    return [total_reward, greedy_warehouse_states]

### Test code

In [17]:
def get_new_warehouse(warehouse, pos, action, color):
    new_warehouse = copy.deepcopy(warehouse)
    if action == 'restore' and warehouse[pos] == color:
        new_warehouse[pos] = 'empty'
    elif action == 'store' and warehouse[pos] == 'empty':
        new_warehouse[pos] = color
    return new_warehouse

In [18]:
def get_state_index(warehouse):
    for i, row in enumerate(state_matrix):
        if list(row) == warehouse:
            return i
        
    return 'failure'

In [19]:

def evaluate_performance(policy, data):
    
    warehouse = ['empty', 'empty','empty', 'empty','empty']
    next_cell_position = 0
    new_warehouse_state_list = list()
    
    for i, row in data.iterrows():
        warehouse[len(cell_index)] = row.action+"_"+row.color
        state_index = get_state_index(warehouse)
        if state_index == 'failure':
            print('Failure:')
            return 'failure'
        else:
            next_cell_position += distance_costs[policy[state_index]] * 2
            warehouse = get_new_warehouse(warehouse, policy[state_index], row.action, row.color)
            new_warehouse_state_list.append(tuple(warehouse))
    return [next_cell_position, new_warehouse_state_list]

### Markov Decision Process Toolbox for policy generation

In [20]:
mdpPolicy = mdptoolbox.mdp.PolicyIteration(tpm_matrix, reward_matrix, 0.3, max_iter=100)
mdpValue = mdptoolbox.mdp.ValueIteration(tpm_matrix, reward_matrix, 0.3, max_iter=100)
mdpPolicy.run()
mdpValue.run()

#### Evaluation on Training data set

In [21]:
mdp_result_policy_results_training = evaluate_performance(mdpPolicy.policy, warehouse_training)
mdp_result_value_results_training = evaluate_performance(mdpValue.policy, warehouse_training)

In [22]:
greedy_distance(warehouse_order)

[228,
 [('red', 'empty', 'empty', 'empty', 'store', 'red'),
  ('red', 'white', 'empty', 'empty', 'store', 'white'),
  ('red', 'empty', 'empty', 'empty', 'restore', 'white'),
  ('red', 'red', 'empty', 'empty', 'store', 'red'),
  ('red', 'red', 'blue', 'empty', 'store', 'blue'),
  ('red', 'red', 'blue', 'red', 'store', 'red'),
  ('empty', 'red', 'blue', 'red', 'restore', 'red'),
  ('empty', 'empty', 'blue', 'red', 'restore', 'red'),
  ('empty', 'empty', 'blue', 'empty', 'restore', 'red'),
  ('red', 'empty', 'blue', 'empty', 'store', 'red'),
  ('red', 'empty', 'empty', 'empty', 'restore', 'blue'),
  ('empty', 'empty', 'empty', 'empty', 'restore', 'red'),
  ('blue', 'empty', 'empty', 'empty', 'store', 'blue'),
  ('empty', 'empty', 'empty', 'empty', 'restore', 'blue'),
  ('red', 'empty', 'empty', 'empty', 'store', 'red'),
  ('red', 'red', 'empty', 'empty', 'store', 'red'),
  ('red', 'red', 'blue', 'empty', 'store', 'blue'),
  ('red', 'red', 'empty', 'empty', 'restore', 'blue'),
  ('empty', 

#### Evaluation on Training data set

In [23]:
greedy_distance(warehouse_training)

[28802,
 [('red', 'empty', 'empty', 'empty', 'store', 'red'),
  ('red', 'red', 'empty', 'empty', 'store', 'red'),
  ('red', 'red', 'red', 'empty', 'store', 'red'),
  ('red', 'red', 'red', 'white', 'store', 'white'),
  ('empty', 'red', 'red', 'white', 'restore', 'red'),
  ('empty', 'empty', 'red', 'white', 'restore', 'red'),
  ('empty', 'empty', 'empty', 'white', 'restore', 'red'),
  ('empty', 'empty', 'empty', 'empty', 'restore', 'white'),
  ('blue', 'empty', 'empty', 'empty', 'store', 'blue'),
  ('blue', 'blue', 'empty', 'empty', 'store', 'blue'),
  ('empty', 'blue', 'empty', 'empty', 'restore', 'blue'),
  ('red', 'blue', 'empty', 'empty', 'store', 'red'),
  ('red', 'blue', 'white', 'empty', 'store', 'white'),
  ('red', 'blue', 'empty', 'empty', 'restore', 'white'),
  ('red', 'blue', 'red', 'empty', 'store', 'red'),
  ('empty', 'blue', 'red', 'empty', 'restore', 'red'),
  ('empty', 'blue', 'empty', 'empty', 'restore', 'red'),
  ('empty', 'empty', 'empty', 'empty', 'restore', 'blue'),


#### Evaluation on Training data set

In [24]:
print("ValueIteration Robot's travel distance: ", mdp_result_policy_results_training[0])
print("PolicyIteration Robot's travel distance: ", mdp_result_value_results_training[0])
print("Greedy Robot's travel distance: ", greedy_distance(warehouse_training)[0])


ValueIteration Robot's travel distance:  28802
PolicyIteration Robot's travel distance:  28802
Greedy Robot's travel distance:  28802


In [25]:
print("ValueIteration Robot's travel distance: ", mdp_result_policy_results_training[1])


ValueIteration Robot's travel distance:  [('red', 'empty', 'empty', 'empty', 'store_red'), ('red', 'red', 'empty', 'empty', 'store_red'), ('red', 'red', 'red', 'empty', 'store_red'), ('red', 'red', 'red', 'white', 'store_white'), ('empty', 'red', 'red', 'white', 'restore_red'), ('empty', 'empty', 'red', 'white', 'restore_red'), ('empty', 'empty', 'empty', 'white', 'restore_red'), ('empty', 'empty', 'empty', 'empty', 'restore_white'), ('blue', 'empty', 'empty', 'empty', 'store_blue'), ('blue', 'blue', 'empty', 'empty', 'store_blue'), ('empty', 'blue', 'empty', 'empty', 'restore_blue'), ('red', 'blue', 'empty', 'empty', 'store_red'), ('red', 'blue', 'white', 'empty', 'store_white'), ('red', 'blue', 'empty', 'empty', 'restore_white'), ('red', 'blue', 'red', 'empty', 'store_red'), ('empty', 'blue', 'red', 'empty', 'restore_red'), ('empty', 'blue', 'empty', 'empty', 'restore_red'), ('empty', 'empty', 'empty', 'empty', 'restore_blue'), ('blue', 'empty', 'empty', 'empty', 'store_blue'), ('blu

#### Evaluation on test data set

In [26]:
mdp_result_policy_results_order = evaluate_performance(mdpPolicy.policy, warehouse_order)
mdp_result_value_results_order = evaluate_performance(mdpValue.policy, warehouse_order)

In [27]:
print("ValueIteration Robot's travel distance: ", mdp_result_policy_results_order[0])
value_iter_states = mdp_result_policy_results_order[1]
print("PolicyIteration Robot's travel distance: ", mdp_result_value_results_order[0])
policy_iter_states = mdp_result_value_results_order[1]
print("Greedy Robot's travel distance: ", greedy_distance(warehouse_order)[0])


ValueIteration Robot's travel distance:  228
PolicyIteration Robot's travel distance:  228
Greedy Robot's travel distance:  228


### Evaluation of training and order data set

In [28]:
print("Warehousetraining 2x2: ")
print("PolicyIteration Robot's travel distance: ", mdp_result_policy_results_training[0])
print("ValueIteration Robot's travel distance: ", mdp_result_value_results_training[0])
print("Greedy Robot's travel distance: ", greedy_distance(warehouse_training)[0])
print("\n")
print("Warehouseorder 2x2: ")
print("PolicyIteration Robot's travel distance: ", mdp_result_policy_results_order[0])
value_iter_states = mdp_result_policy_results_order[1]
print("ValueIteration Robot's travel distance: ", mdp_result_value_results_order[0])
policy_iter_states = mdp_result_value_results_order[1]
print("Greedy Robot's travel distance: ", greedy_distance(warehouse_order)[0])

Warehousetraining 2x2: 
PolicyIteration Robot's travel distance:  28802
ValueIteration Robot's travel distance:  28802
Greedy Robot's travel distance:  28802


Warehouseorder 2x2: 
PolicyIteration Robot's travel distance:  228
ValueIteration Robot's travel distance:  228
Greedy Robot's travel distance:  228


In [29]:
print("Greedy Robot's policy: ", greedy_distance(warehouse_order)[1][0:14])
print("\n")
print("PolicyIteration Robot's policy: ", mdp_result_policy_results_order[1])


Greedy Robot's policy:  [('red', 'empty', 'empty', 'empty', 'store', 'red'), ('red', 'white', 'empty', 'empty', 'store', 'white'), ('red', 'empty', 'empty', 'empty', 'restore', 'white'), ('red', 'red', 'empty', 'empty', 'store', 'red'), ('red', 'red', 'blue', 'empty', 'store', 'blue'), ('red', 'red', 'blue', 'red', 'store', 'red'), ('empty', 'red', 'blue', 'red', 'restore', 'red'), ('empty', 'empty', 'blue', 'red', 'restore', 'red'), ('empty', 'empty', 'blue', 'empty', 'restore', 'red'), ('red', 'empty', 'blue', 'empty', 'store', 'red'), ('red', 'empty', 'empty', 'empty', 'restore', 'blue'), ('empty', 'empty', 'empty', 'empty', 'restore', 'red'), ('blue', 'empty', 'empty', 'empty', 'store', 'blue'), ('empty', 'empty', 'empty', 'empty', 'restore', 'blue')]


PolicyIteration Robot's policy:  [('red', 'empty', 'empty', 'empty', 'store_red'), ('red', 'white', 'empty', 'empty', 'store_white'), ('red', 'empty', 'empty', 'empty', 'restore_white'), ('red', 'red', 'empty', 'empty', 'store_red')

## Literature

[1] https://www.youtube.com/watch?v=FgzM3zpZ55o&list=PLoROMvodv4rOSOPzutgyCTapiGlY2Nd8u